# A hands-on guide to LLM-based AI Agents
# Exercise Block 1
#### ML Prague 2025
#### Philipp Wendland <pwendland@deloitte.de>
This notebook contains exercises for the workshop at ML Prague 2025. The goal is to implement an AI Agent that can assist you in exploring a new city and planning various trips according to your preferences.
***
###### Sources: 
- https://huggingface.co/blog/smolagents
- https://huggingface.co/docs/smolagents/index
- https://huggingface.co/learn/agents-course/
- https://github.com/anthropics/anthropic-cookbook/
- https://platform.openai.com/docs/overview
***

## Exercise 1: Simple Prompt
This exercise will ensure your environment, API keys and accesses are configured correctly. Additionally you will create a simple prompt to use as a reference point when building your agent.

Complete the following items:
- [ ] Set up API key
- [ ] Adjust the prompt to your preferences
- [ ] Examine the simple output from the LLM

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Either enter your API key directly here
# client = OpenAI(api_key="***")
# or use dotenv to set it as an environment variable (or simply export it)
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

model = "gpt-4o-mini"
system_prompt = "Be a helpful assistant"
city = "Prague"

prompt = "Act as a professional travel planner. I'm staying in {0} for the Machine Learning Prague conference.\
During the day I will attend seminars and talks, but in the evening I'm mostly free. Plan an itinerary for must-see things\
in {0} for Monday and Tuesday night. I'm interested in both night-life and culture.".format(city)

def llm_call_openai(prompt: str, model="gpt-4o-mini") -> str: 
    """
        Calls the specified OpenAI model with the given prompt
        Returns: 
            str: Completion from the LLM
    """

    response = client.responses.create(
        model=model,
        instructions=system_prompt,
        input=prompt
    )

    return response.output[0].content[0].text

completion = llm_call_openai(prompt, model)

print(completion)

## Exercise 2: Prompt Workflow
Create a prompt workflow (aka prompt-chain) to break down the task of planning an itinerary into multiple steps. The `sequential_chain` method always passes the output of the previous generation between steps. In the first step `add_information` is passed.

Complete the following items:
- [ ] Edit the list of prompts `create_itinerary_prompts`, add at least one more sequential prompt. Compare the output to the results from Exercise 1
- [ ] Write the `additive_chain` function, where all previous generated output is passed between steps, change the prompt-list if necessary and re-run the chain
- [ ] **BONUS**: Try a different use case: e.g., create a marketing pitch for the sight-seeing tour and translate it to a different language / tonality.

In [ ]:
def sequential_chain(prompts, input = "") -> str:
    """Create a sequential chain of prompts, where only the result is passed between steps."""
    completion = input
    for i, prompt in enumerate(prompts, 1):
        print("Step {}:".format(i))
        completion = llm_call_openai("{}\n###\nGiven information: {}\n###\n".format(prompt, completion))
        print(completion)
    return completion


# @TODO: Adjust the prompts to your preferences
create_itinerary_prompts = [
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative.""",
    
    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night.""",
]

add_information = "I'm interested in both night-life and culture."

sequential_chain(create_itinerary_prompts, add_information)


In [ ]:
def additive_chain(prompts, input = "") -> str:
    """Create an additive chain of prompts, where all previous input and output is passed between steps."""
    
    history = ""
    # @TODO: Implement the additive_chain function
    # YOUR CODE
    # END 
    return history
    
# @TODO: Adjust the prompts to your preferences
create_itinerary_prompts_additive = [
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative""",
    
    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""
]

print("\n\nComplete interaction\n{}".format(additive_chain(create_itinerary_prompts_additive, add_information)))

**Bonus**: Try a different use case: e.g., create a marketing pitch for the sight-seeing tour and translate it to a different language / tonality.

In [ ]:
marketing_pitch_prompts = [
    # @TODO: Edit the prompts
    """Prompt 1""",
    
    """Prompt 2""",

    """Prompt 3""",

    """Prompt 4"""
]

print(sequential_chain(marketing_pitch_prompts))


## Exercise 3a: Tool Calling
Before introducing the smolagents framework, we will implement a tool call using standard OpenAI functionalities. We will add the option for tool calls into the prompt-chain.

- [ ] Examine the implemented tool call for obtaining current weather information.
    - [ ] Extend both `llm_call_openai` and `sequential_chain` to be able to handle tool calls
    - [ ] Add the tool call in the beginning of the prompt chain to add current weather information to the planning.

In [ ]:
# For additional documentation refer to: https://platform.openai.com/docs/guides/function-calling#sample-function
# open-meteo docs: https://open-meteo.com/en/docs
import requests
import json

def get_current_weather(latitude=50.073658, longitude=14.418540):
    try: 
        response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,cloud_cover,precipitation&hourly=temperature_2m,precipitation_probability,cloud_cover")
    except: 
        return "Weather data not available"
    response_data = response.json()
    return {
        "current_temp": response_data['current']['temperature_2m'],
        "cloud_cover": response_data['current']['cloud_cover'],
        "precipitation": response_data['current']['precipitation']
    }


tools = [{
    "type": "function",
    "name": "get_current_weather",
    "description": "Get current weather for provided coordinates, including the temperature in celsius, the cloud cover percentage and the precipitation in mm.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

prompt = "What is the weather like in Melbourne today?"

response = client.responses.create(
    model="gpt-4o-mini",
    input=prompt,
    tools=tools,
)
print(response)
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

current_weather = get_current_weather(args['latitude'], args["longitude"])

print("\nTool output:\n{}".format(current_weather))

In [ ]:
# Helper function to accomodate for multiple tool calls
def call_function(name, args): 
    if name == "get_current_weather":
        return get_current_weather(**args)


def llm_call_openai_tools(prompt: str, tools=[], model="gpt-4o-mini") -> str: 
    """
        Calls the specified OpenAI model with the given prompt and if specified uses the given tools
        Returns: 
            str: Completion from the LLM or result from the tool call
    """

    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    if not tools: 
        # YOUR CODE
    else:
        
        # YOUR CODE
        
        return tool_call_responses

def sequential_chain_with_tools(prompts, input = "") -> str:
    """Create a sequential chain of prompts, where only the result is passed between steps.
    Prompts now is a list of tuples that can contain a list of tools"""
    completion = input
    for i, prompt in enumerate(prompts, 1):
        print("Step {}:".format(i))

        # YOUR CODE
        
    return completion

# Adjust the prompts to your preferences
create_itinerary_prompts_with_tools = [
    ["""What is the current weather in Prague""", tools],
    
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative""",

    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""
]

sequential_chain_with_tools(create_itinerary_prompts_with_tools)